In [1]:
import csv
import os
import numpy as np
import pandas as pd
import sklearn.metrics as mm
import sys
# sir_inference imports
from sir_model import FastProximityModel, patient_zeros_states
from ranking import csr_to_list
import os.path
from os import path
N=50000
## new try with 100 spreaders
N_patient_zero = 20;
lamb = 0.03;
mu = 0.01;
scale=1.0; # Easy Case
T=100;
seed=1
#seed=int();
print("Generate network with N=%d T=%d scale=%.1f default lambda=%.2f seed=%d..."%(N,T,scale,lamb,seed), flush=True)
initial_states = patient_zeros_states(N, N_patient_zero)
model = FastProximityModel(N, scale, mu, lamb, initial_states)
location="networks"
if path.isdir(location) : print("Will save in "+location)
else : 
    print("log file not found. was looking for: \n "+location+"\n Bye Bye")
    sys.exit()
model.run(T=T, print_every=20)
# model.get_counts().plot(
#     title=f"N_patient_zero={N_patient_zero} lamb={lamb:.2f}  mu={mu:.2f}"
# );
print("Saving transmissions...", flush=True)
logfile="interactions_proximity_N%dK_s%.1f_T%d_lamb%.2f_s%d.csv"%(N/1000,scale,T,lamb,seed)
with open(location+"/"+logfile, 'w', newline='') as csvfile:
    fieldnames = ['t', 'i', 'j', 'lamb']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for t, A in enumerate(model.transmissions):
        for i, j, lamb in csr_to_list(A):
            writer.writerow(dict(t=t, i=i, j=j, lamb=lamb))
print("Bye-Bye")

Generate network with N=50000 T=100 scale=1.0 default lambda=0.03 seed=1...
Will save in networks
Generating transmissions
t = 0 / 100
t = 20 / 100
t = 40 / 100
t = 60 / 100
t = 80 / 100
Running SIR simulation
t = 0 / 100
t = 20 / 100
t = 40 / 100
t = 60 / 100
t = 80 / 100
Saving transmissions...
Bye-Bye


In [2]:
import sys
from time import time
import numpy as np
import pandas as pd
import os.path
from os import path
from scenario import Scenario
from sir_model import EpidemicModel, patient_zeros_states, symptomatic_individuals
from ranking import RANKINGS

# SIR parameters
location="networks"
t1 = 10;
# trac parameters
trac_tau = 5;
# MF parameters
MF_taus = [5];
MF_deltas = [15];
# observation parameters
n_rankings = [50]
#seed = int(sys.argv[2]);
#seeds for running [32,123,456]
#seed=int(sys.argv[1]);
################################################
for MF_tau in MF_taus:
    for MF_delta in MF_deltas:
        print("Start : No App on Proximity Newtork", flush=True)
        for n_ranking in n_rankings:
            log_fname="interactions_proximity_N%dK_s%.1f_T%d_lamb%.2f_s%d.csv"%(N/1000,scale,T,lamb,seed)
            csv_file=location+"/"+log_fname
            print("seed = %d"%seed, flush=True)
            np.random.seed(seed);
            if path.exists(csv_file): print("Loading model from %s"%csv_file)
            else:
                print("Could not find the file! Was looking for \n "+csv_file+"\n Bye-Bye")
                sys.exit()
            print("Load Proximity model", flush=True)
            tic = time()
            model = EpidemicModel(initial_states=np.zeros(N), x_pos=np.zeros(N), y_pos=np.zeros(N))
            model.load_transmissions(csv_file)
            model.recover_probas = mu*np.ones(N)
            print(f"Loading took {time()-tic:.1f}s",flush=True)
            model.initial_states = patient_zeros_states(N, N_patient_zero)
            model.time_evolution(model.recover_probas, model.transmissions, print_every=50)
            t_max = len(model.transmissions)
            print("Save plain dynamics", flush=True)
            db = pd.DataFrame()
            db["S"] = np.sum(model.states==0,axis=1)
            db["I"] = np.sum(model.states==1,axis=1)
            db["R"] = np.sum(model.states==2,axis=1)
            db.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed),
                      index=False, sep="\t")
            del db
            model.initial_states = model.states[t1]
            model.states = model.states[t1:]
            model.transmissions = model.transmissions[t1:]
            intervention_options=dict(quarantine_time=50)
            observation_options=dict(n_random=0,n_infected=0,n_ranking=n_ranking, p_symptomatic=0.5, tau=5)
            scenario_rnd = Scenario(
                model, seed=seed, 
                ranking_options=dict(ranking=RANKINGS["random"]),
                observation_options=observation_options,
                intervention_options=intervention_options
            )
            scenario_rnd.run(t_max-t1)
            print("Save random strategy", flush=True)
            scenario_rnd.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_rnd.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking),
                      index=False, sep="\t")
            del scenario_rnd
            for p_untracked in [0]:
                observation_options=dict(n_random=0,n_infected=0,n_ranking=n_ranking, p_symptomatic=0.5, tau=5, p_untracked=p_untracked)
                scenario_trac = Scenario(    model, seed=seed, 
                    ranking_options=dict(ranking=RANKINGS["tracing"], tau=5),
                    observation_options=observation_options,
                    intervention_options=intervention_options
                )
                scenario_trac.run(t_max-t1)
                print("Save tracing strategy", flush=True)
                if p_untracked==0:
                    scenario_trac.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_trac_t%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau),
                              index=False, sep="\t")
                else:
                    scenario_trac.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_noapp%.2f_trac_t%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,p_untracked,trac_tau),
                              index=False, sep="\t")
                del scenario_trac
                scenario_MF = Scenario(
                    model, seed=seed, 
                    ranking_options=dict(ranking=RANKINGS["backtrack"], algo="MF", init="all_S", tau=5, delta=10), 
                    observation_options=observation_options,
                    intervention_options=intervention_options
                )
                scenario_MF.run(t_max-t1)
                print("Save MF strategy", flush=True)
                if p_untracked==0:
                    scenario_MF.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_MF_t%d_d%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,MF_tau,MF_delta),
                              index=False, sep="\t")
                else:
                    scenario_MF.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_noapp%.2f_MF_t%d_d%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,p_untracked,MF_tau,MF_delta),
                              index=False, sep="\t")
                del scenario_MF
            # 1h01min per round
print("End seed", flush=True)
#del model

Start : No App on Proximity Newtork
seed = 1
Loading model from networks/interactions_proximity_N50K_s1.0_T100_lamb0.03_s1.csv
Load Proximity model
Loading transmissions from networks/interactions_proximity_N50K_s1.0_T100_lamb0.03_s1.csv
Loading took 28.8s
Running SIR simulation
t = 0 / 100
t = 50 / 100
Save plain dynamics
Using ranking_random to rank
Using seed=1
Selecting 0/50000 untracked individuals
Save random strategy
Using ranking_tracing to rank
Using seed=1
Selecting 0/50000 untracked individuals
Save tracing strategy
Using ranking_backtrack to rank
Using seed=1
Selecting 0/50000 untracked individuals
Save MF strategy
End seed


In [7]:
import sys
sys.path.insert(0,'../loop_abm/src/')
import abm_utils
import sib
import scipy
from loop_ranker import sib_rank, sib_drop_rank, greedy_rank, dotd_rank, mean_field_rank, tracing_rank, winbp_rank, winbp_prob0_rank
pseed=10/N
psus = 0.52

class dummy_logger():
    def info(self, x):
        print(x)
        
class bp_ranker_class(winbp_prob0_rank.WinBPProb0Ranker):
    
    def step_scenario(self, t, model, observations, params):
        data = {}
        data["logger"] = dummy_logger()
        daily_contacts = [(i, j, t, l) for i, j, l in list(csr_to_list(model.transmissions[t]))]
        daily_obs = []
        print("SCENARIO BP")
        print(t, "obs", len(observations))
        if len(observations)!=0:
            observations = pd.DataFrame(observations)
            #print(observations)
            observations = observations[observations["t_test"] == t-1]
            #print(t, observations)
            daily_obs = list(observations[["i", "s", "t_test"]].itertuples(index=False, name=None))  
        #print(t, daily_contacts[0])
        #print(t, daily_obs)
        rank_algo = self.rank(t, daily_contacts, daily_obs, data)
        rank = sorted(rank_algo, key= lambda tup: tup[1], reverse=True)
        rank = [(int(tup[0]), tup[1])  for tup in rank]
        rank_pd = pd.DataFrame(rank, columns=["i", "score"])
        rank_pd["rank"] = range(model.N)
        #print(rank_pd)
        return rank_pd

In [15]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
observation_options=dict(n_random=0,n_infected=0,n_ranking=n_ranking, p_symptomatic=0.5, tau=5, p_untracked=p_untracked)
name_csv = "csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_bp%d.csv"%(N, T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau
bp_ranker = bp_ranker_class(params = sib.Params(
                                 prob_r = sib.Exponential(mu=1/30),
                                 pseed = pseed,
                                 psus = psus),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-2,
                 window_length = 20,
                 tau = 7)
bp_ranker.init(N, T)
bp_ranker.__name__ = "bp"
scenario_trac = Scenario( model, seed=seed, 
    ranking_options=dict(ranking=bp_ranker.step_scenario),
    observation_options=observation_options,
    intervention_options=intervention_options,
                         name_csv = name_csv
)
scenario_trac.run(t_max-t1, print_every = 1)
print("Save bp strategy", flush=True)
scenario_trac.counts.to_csv(name_csv,
          index=False, sep="\t")


Using step_scenario to rank
Using seed=1
Selecting 0/50000 untracked individuals
SCENARIO BP
0 obs 0
winBP: (S,I,R): (49981.0, 18.3, 0.6), seeds: 19.0, ll: -32677.4
here
t = 0 / 90
inside print every
        S   I  R  q
t                  
0   49912  83  5  0
1   50000   0  0  0
2   50000   0  0  0
3   50000   0  0  0
4   50000   0  0  0
..    ...  .. .. ..
85  50000   0  0  0
86  50000   0  0  0
87  50000   0  0  0
88  50000   0  0  0
89  50000   0  0  0

[90 rows x 4 columns] csv/temp_scenario.csv


AttributeError: 'Scenario' object has no attribute 'count'

In [5]:
bp_ranker = bp_ranker_class(params = sib.Params(
                                 prob_r = sib.Exponential(mu=1/30),
                                 pseed = pseed,
                                 psus = psus),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-2,
                 window_length = 20,
                 tau = 7)
bp_ranker.init(N, T)

In [ ]:
bp_ranker.rank()

In [41]:
a = model.transmissions[0]

In [60]:
from ranking import RANKINGS, csr_to_list


In [62]:
a = csr_to_list(model.transmissions[0])

In [67]:
t = 0
daily_contacts = [(i, j, t, l) for i, j, l in list(csr_to_list(model.transmissions[t]))]

In [69]:
len(daily_contacts)

5406

In [59]:
model.transmissions[0].toarray().shape

(1000, 1000)

In [33]:
model.recover_probas

array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.

In [21]:
bp_ranker

__main__.bp_ranker

In [16]:
T, N

(100, 1000)

In [17]:
winbp20prob0_tau7.init(T, N)

In [ ]:
scenario_BP = Scenario(
                    model, seed=seed, 
                    ranking_options=dict(ranking=RANKINGS["backtrack"], algo="MF", init="all_S", tau=5, delta=10), 
                    observation_options=observation_options,
                    intervention_options=intervention_options
                )